In [1]:
import pandas as pd
import cenpy as cen
import pysal
import requests
import numpy as np

/Users/ashleywhite/anaconda3/lib/python3.6/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


In [2]:
datasets = list(cen.explorer.available(verbose=True).items())

datasets = pd.DataFrame(datasets, columns = ['code', 'dataset name'])

In [3]:
qwi_connection = cen.base.Connection('QWISA', apikey = 'c32701ea1121c5cc00309985cc54806ded623d93')
qwi_connection

Connection to Time Series Longitudinal Employer-Household Dynamics - QWI: Sex by Age (ID: http://api.census.gov/data/id/QWISA)

In [4]:
g_unit = 'county:*'
g_filter = {'state': '02'}
time = 'from 2003-Q1 to 2018-Q1'

In [5]:
# Create State Codes
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

states = list(state_codes.values())

In [6]:
cols = qwi_connection.varslike('Emp')
hir = qwi_connection.varslike('HirA')
earns = qwi_connection.varslike('Earn')
payroll = qwi_connection.varslike('Pay')
firm = qwi_connection.varslike('Frm')
sep = qwi_connection.varslike('sep')

cols.extend(hir)
cols.extend(earns)
cols.extend(payroll)
cols.extend(firm)
cols.extend(sep)

In [7]:
sectors = [111, 112, 113, 114, 115, 211, 212, 213, 221, 236, 237, 238, 
            311, 312, 313, 314, 315, 316, 321, 322, 323, 324, 325, 326, 327, 331, 332, 333, 334, 335, 336, 337, 339, 
            423, 424, 425, 441, 442, 443, 444, 445, 446, 447, 448, 451, 452, 453, 454, 481, 482, 483, 484, 485, 486, 487, 488, 491, 492, 493, 
            511, 512, 515, 517, 518, 519, 521, 522, 523, 524, 525, 531, 532, 533, 541, 551, 561, 562, 
            611, 621, 622, 623, 624, 
            711, 712, 713, 721, 722, 
            811, 812, 813, 814, 
            921, 922, 923, 924, 925, 926, 927, 928
             ]

In [8]:
industries = [11, 21, 22, 23, 31, 32, 33, 42, 44, 45, 48, 49, 51, 52, 53, 54, 55, 56, 61, 62, 71, 72, 81, 92]

In [9]:
master = qwi_connection.query(cols = cols, industry = '11',
                              time = time, geo_filter = g_filter, geo_unit = g_unit,
                         )

/Users/ashleywhite/anaconda3/lib/python3.6/site-packages/cenpy/remote.py:172: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df[cols] = df[cols].convert_objects(convert_numeric=convert_numeric)


In [10]:
master.head()

,EmpSpv,Emp,EmpTotal,EmpEnd,EmpS,HirAEnd,HirAEndR,HirAs,HirAEndRepl,HirAEndReplr,...,FrmJbCS,FrmJbC,FrmJbGn,FrmJbGnS,FrmJbLsS,FrmJbLs,industry,time,state,county
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,2003-Q1,02,013
1,13.0,14.0,14.0,11.0,11.0,0.0,0.000,0.0,0.0,0.000,...,-2.0,-3.0,0.0,0.0,2.0,3.0,11,2003-Q1,02,016
2,109.0,133.0,180.0,139.0,103.0,36.0,0.265,12.0,21.0,0.154,...,-7.0,3.0,15.0,1.0,8.0,12.0,11,2003-Q1,02,020
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,2003-Q1,02,050
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,2003-Q1,02,070


In [13]:
for s in states:  
    
    print(f'Scraping {s}')
    
    for x in industries:
        
        try:
            g_unit = 'county:*'
            g_filter = {'state': s}
            time = 'from 2003-Q1 to 2018-Q1'

            
            df = qwi_connection.query(cols = cols, 
                                      #industry = x,
                                      time = time, geo_filter = g_filter, geo_unit = g_unit, 
                                  )

            master = pd.concat([master, df])
            
        except requests.exceptions.HTTPError:
            pass

Scraping 53


/Users/ashleywhite/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


Scraping 10
Scraping 11
Scraping 55
Scraping 54
Scraping 15
Scraping 12
Scraping 56
Scraping 34
Scraping 35
Scraping 48
Scraping 22
Scraping 37
Scraping 38
Scraping 31
Scraping 47
Scraping 36
Scraping 42
Scraping 02
Scraping 32
Scraping 33
Scraping 51
Scraping 08
Scraping 06
Scraping 01
Scraping 05
Scraping 50
Scraping 17
Scraping 13
Scraping 18
Scraping 19
Scraping 25
Scraping 04
Scraping 16
Scraping 09
Scraping 23
Scraping 24
Scraping 40
Scraping 39
Scraping 49
Scraping 29
Scraping 27
Scraping 26
Scraping 44
Scraping 20
Scraping 30
Scraping 28
Scraping 45
Scraping 21
Scraping 41
Scraping 46


In [14]:
master.to_csv('../project-4/data/master_qwi_2003_2018.csv')

In [15]:
master.head()

,EarnBeg,EarnHirAS,EarnHirNS,EarnS,EarnSepS,Emp,EmpEnd,EmpS,EmpSpv,EmpTotal,...,HirAEnd,HirAEndR,HirAEndRepl,HirAEndReplr,HirAs,Payroll,county,industry,state,time
0,1744.0,NaN,NaN,1744.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,5370,013,11,02,2003-Q1
1,14499.0,NaN,NaN,16262.0,4647.0,14.0,11.0,11.0,13.0,14.0,...,0.0,0.000,0.0,0.000,0.0,570042,016,11,02,2003-Q1
2,1863.0,1455.0,1672.0,2134.0,1966.0,133.0,139.0,103.0,109.0,180.0,...,36.0,0.265,21.0,0.154,12.0,875806,020,11,02,2003-Q1
3,4917.0,8740.0,NaN,8740.0,8740.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,89678,050,11,02,2003-Q1
4,3234.0,NaN,NaN,3234.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,9955,070,11,02,2003-Q1


In [16]:
master.shape

(4246815, 27)

In [23]:
master.isnull().mean().sort_values(ascending = False)

industry        0.999775
EarnSepS        0.034565
EarnHirNS       0.021992
FrmJbGnS        0.018763
FrmJbCS         0.018763
FrmJbLsS        0.018763
EarnHirAS       0.018570
HirAs           0.018483
HirAEndReplr    0.018091
HirAEndR        0.018090
HirAEndRepl     0.018082
FrmJbLs         0.018081
FrmJbGn         0.018081
FrmJbC          0.018081
HirAEnd         0.018074
EmpS            0.018065
EarnS           0.017393
EmpEnd          0.017378
EmpSpv          0.002102
Emp             0.001415
HirA            0.001030
EmpTotal        0.000736
EarnBeg         0.000713
state           0.000000
Payroll         0.000000
county          0.000000
time            0.000000
dtype: float64